In [1]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np
import seaborn as sns
import plotly.express as px
import torch
import os
from tqdm import tqdm
import pickle
from Levenshtein import distance as lev_distance

import warnings

warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize': (20, 10), 'figure.facecolor': 'white'})
sns.set_palette("viridis")
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pandarallel.initialize(progress_bar=True)
os.environ["TOKENIZERS_PARALLELISM"] = "true"  # activate parallelism

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Stage 1 - Get and preprocess Data

In [2]:
def join_non_null_values(l):
    res = [str(x) for x in l if str(x) != 'nan']
    return ', '.join(res)

In [18]:
train = (
    pd.read_csv('../data/raw/additional_data/building_20230808.csv').add_suffix('_building')
    .merge(
        pd.read_csv('../data/raw/additional_data/district_20230808.csv').add_suffix('_district'),
        left_on='district_id_building',
        right_on='id_district',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/prefix_20230808.csv').add_suffix('_prefix'),
        left_on='prefix_id_building',
        right_on='id_prefix',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/town_20230808.csv').add_suffix('_town'),
        left_on='town_id_prefix',
        right_on='id_town',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonim_20230808.csv').add_suffix('_geonim'),
        left_on='geonim_id_prefix',
        right_on='id_geonim',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonimtype_20230808.csv').add_suffix('_geonimtype'),
        left_on='type_id_geonim',
        right_on='id_geonimtype',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/area_20230808.csv').add_suffix('_area'),
        left_on='area_id_prefix',
        right_on='id_area',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/areatype_20230808.csv').add_suffix('_areatype'),
        left_on='type_id_area',
        right_on='id_areatype',
        how='left',
    )
    .assign(name_district_full=lambda df_: df_.name_district + ' район')
    .assign(
        all_in_field=lambda df_: df_[['full_address_building', 'type_building',
                                      'name_district_full', 'name_area', 'name_areatype']]
        .apply(lambda x: join_non_null_values(x), axis=1)
    )
    .assign(miscellaneous_objects = lambda df_: df_.parallel_apply(lambda x: get_address_ner_objects(x['full_address_building']), axis=1))
    
)
train = pd.concat([train.drop(['miscellaneous_objects'], axis=1), train['miscellaneous_objects'].apply(pd.Series)], axis=1)
train.to_pickle("../data/processed/main_data.pkl")
train

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field,город,улица,дом,поселок,проспект,корпус,Литера,муниципальный район,набережная,шоссе,парк,переулок,площадь,аллея,,линия,автодорога,микрорайон,деревня,проезд,квартал,бульвар,станция,район,территория,муниципальный округ,мост,тупик,область,село,поселок городского типа,округ,волость
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN,35,Пушкинский,True,True,11132,28.0,4801.0,NaN,NaN,"город Пушкин, Кедринская улица","г.Пушкин, Кедринская ул.","'город':1,5 'кедринск':7 'кедринская':3 'пушкин':2,6 'улиц':8 'улица':4",True,True,15.0,True,28.0,город Пушкин,г.Пушкин,"'город':1,3 'пушкин':2,4",True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пушкинский район,"город Пушкин, Кедринская улица, дом 12, Пушкинский район",ПУШКИН,КЕДРИНСКАЯ,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN,38,Курортный,True,True,6987,46.0,3.0,NaN,NaN,"посёлок Ушково, Пляжевая улица","пос. Ушково, Пляжевая ул.",'пляжев':7 'пляжевая':3 'поселок':5 'посёлок':1 'улиц':8 'улица':4 'ушков':6 'ушково':2,True,True,15.0,True,46.0,посёлок Ушково,пос. Ушково,'поселок':3 'посёлок':1 'ушков':4 'ушково':2,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Ушково, Пляжевая улица, дом 4Б, Курортный район",NaN,ПЛЯЖЕВАЯ,4Б,УШКОВО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, к. 2, л. Е",198303,NaN,15,Кировский,True,True,6469,36.0,2194.0,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова","г.Санкт-Петербург, пр. Маршала Жукова","'г':1,8 'жуков':14 'жукова':7 'марша':13 'маршала':6 'петербург':4,11 'проспект':5,12 'санкт':3,10 'санкт-петербург':2,9",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кировский район,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е, Нежилое, Кировский район",САНКТ-ПЕТЕРБУРГ,NaN,30,NaN,МАРШАЛА ЖУКОВА,2,Е,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, дом 5, ко

In [ ]:
train = 

In [19]:
# 'город' 'поселок' 'деревня' 'село' 'поселок городского типа'
# 'улица' 'проспект' 'набережная' 'шоссе' 'парк' 'переулок' 'площадь' 'аллея' 'линия' 'автодорога' 'проезд' 'бульвар'
# 'дом'
# 'корпус'
# 'Литера'
# 'муниципальный район' 'микрорайон' 'квартал' 'район' 'муниципальный округ'
# 'станция' 'территория' 'мост' 'тупик'
#  'область' 'округ' 'волость'
train.columns

Index(['id_building', 'prefix_id_building', 'district_id_building',
       'house_building', 'corpus_building', 'liter_building', 'villa_building',
       'parcel_building', 'full_address_building', 'is_updated_building',
       'is_actual_building', 'type_building', 'municipality_id_building',
       'short_address_building', 'post_prefix_building',
       'build_number_building', 'id_district', 'name_district',
       'is_updated_district', 'is_actual_district', 'id_prefix',
       'town_id_prefix', 'geonim_id_prefix', 'area_id_prefix',
       'toponim_id_prefix', 'name_prefix', 'short_name_prefix',
       'search_index_prefix', 'is_updated_prefix', 'is_actual_prefix',
       'sub_rf_id_prefix', 'has_buildings_prefix', 'id_town', 'name_town',
       'short_name_town', 'search_index_town', 'is_updated_town',
       'is_actual_town', 'has_buildings_town', 'id_geonim', 'type_id_geonim',
       'name_geonim', 'short_name_geonim', 'is_updated_geonim',
       'is_actual_geonim', 'only_name

In [ ]:
# train.rename(columns={'город': 'ner_city', 'улица': 'ner_stree', 'дом': 'ner_house', 'поселок': 'ner_community', 'проспект': 'prospect',})

In [ ]:
pd.concat([train.drop(['miscellaneous_objects'], axis=1), train['miscellaneous_objects'].apply(pd.Series)], axis=1)

In [13]:
train = pd.read_pickle("../data/processed/main_data.pkl")

In [8]:
for_embeddings = (
    pd.DataFrame(
        np.concatenate([
            train[['id_building', 'full_address_building']].values,
            train[['id_building', 'all_in_field']].values,
            # train[['id_building', 'short_address_building']].values,
            # train[['id_building', 'name_prefix']].values,
            # train[['id_building', 'short_name_prefix']].values,
            # train[['id_building', 'name_town']].values,
            # train[['id_building', 'short_name_town']].values,
            # train[['id_building', 'name_geonim']].values,
            # train[['id_building', 'short_name_geonim']].values,
            # train[['id_building', 'only_name_geonim']].values,
            # train[['id_building', 'name_prefix']].values,
            # train[['id_building', 'short_name_prefix']].values,
            # train[['id_building', 'name_geonimtype']].values,
            # train[['id_building', 'only_name_area']].values,
            # train[['id_building', 'name_areatype']].values,
            # train[['id_building', 'short_name_areatype']].values,
            # train[['id_building', 'name_district']].values,
            # train[['id_building', 'name_district_full']].values,
        ], axis=0),
        columns=['id_building', 'address']
    )
    .dropna()
    .reset_index(drop=True)
)

In [10]:
for_embeddings.to_pickle('../data/processed/for_embeddings_with_names.pkl')
for_embeddings

,id_building,address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е"
3,124415,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2"
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"
...,...,...
333285,18941,"г.Санкт-Петербург, Бородинская улица, дом 3, литера В, Нежилое, Адмиралтейский район"
333286,18942,"посёлок Александровская, 2-я линия, дом 34/3, литера А, Жилое, Пушкинский район"
333287,18943,"посёлок Ушково, Детская улица, дом 39, Курортный район"
333288,18944,"г.Санкт-Петербург, Заусадебная улица, дом 25, литера Б, Нежилое, Приморский район"


# Step 2 - Get Embeddings

In [124]:
from sentence_transformers import SentenceTransformer #, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# embeddings_list = [0] * len(for_embeddings)

In [12]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [13]:
# embeddings = tokenizer(list(for_embeddings['address']), padding=True, truncation=True, return_tensors='pt')

In [126]:
embeddings = model.encode(for_embeddings['address'].astype('str'), convert_to_tensor=True, show_progress_bar=True,
                          batch_size=128)

In [24]:
with open('../data/processed/embeddings.pkl', 'wb') as fp:
    pickle.dump(embeddings, fp)

In [25]:
# embeddings.to_pickle("../data/processed/embeddings.pkl")

In [160]:
# #split DataFrame into chunks and iter over it creating embeddings
# n = 1000 #specify number of rows in each chunk
# embeddings = torch.empty(0)
# for batch in tqdm([for_embeddings[i:i + n] for i in range(0, len(for_embeddings), n)]):
#     batch_embedding = model.encode(batch['address'].astype('str'), convert_to_tensor=True, show_progress_bar=True)
#     embeddings = torch.concat([embeddings, batch_embedding])

In [161]:
# batch

In [ ]:
# for_embeddings['address'].parallel_apply(lambda x: model.encode(x, convert_to_tensor=True))

In [162]:

# embeddings_list = []
# for indx in tqdm(range(len(for_embeddings))):
# # train_emb_only.loc[indx, 'full_address_embedding'] = Fake(model.encode(train_emb_only.loc[indx, 'full_address'], convert_to_tensor=True))
#     embeddings_list.append(model.encode(for_embeddings.loc[indx, 'address'], convert_to_tensor=True))
# # train_emb_only.full_address_embedding = train_emb_only.full_address_embedding.apply(lambda x: x.obj)
# embeddings_list

# Step 3 - Similarity Matcher

In [134]:
with open('../data/processed/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

for_embeddings = pd.read_pickle("../data/processed/for_embeddings_with_names.pkl")

In [203]:
QUERY = ["г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М"]

In [208]:
def get_best_matches(query, top_n, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = embeddings.dot(query_embedding.T).ravel()
    best = np.argpartition(scores, -top_n)[-top_n:]
    sentences['scores'] = scores
    sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'address'].apply(
        lambda x: lev_distance(query, x))
    best_matches = sentences[['id_building', 'address', 'scores', 'lev_distance']].loc[list(best)]
    best_matches_within_id_indices = best_matches.groupby(['id_building']).scores.transform(max) == best_matches.scores
    best_matches = best_matches.loc[best_matches_within_id_indices].sort_values(by='scores',
                                                                                ascending=False).reset_index(drop=True)
    if len(best_matches[best_matches['scores'] >= 0.99]):
        return best_matches[best_matches['scores'] >= 0.99]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    else:
        return best_matches
    # return best_matches.loc[best_matches_within_id_indices].sort_values(by='lev_distance').reset_index(drop=True)

In [205]:
get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance
0,84908,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",1.0,65.0


In [209]:
QUERY = ["г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",
         'г.Санкт-Петербург, набережная Обводного канала, дом 205, литера А']

def multiple_best_matches(file, top_n, embeddings: np.array, sentences):
    res = pd.DataFrame()
    for el in file:
        res = pd.concat([res, get_best_matches(el, top_n, embeddings, sentences)])
    return res.reset_index(drop=True)
    

In [212]:
x = multiple_best_matches(train['full_address_building'].loc[:10].values, 10, np.array(embeddings), for_embeddings)
x

,id_building,address,scores,lev_distance
0,56343,"город Пушкин, Кедринская улица, дом 12",1.000000,0.0
1,595,"поселок Ушково, Пляжевая улица, дом 4Б",1.000000,0.0
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",1.000000,0.0
3,199215,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",1.000000,0.0
4,39404,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера И",0.991955,1.0
5,39211,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Д",0.991091,1.0
6,61406,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера В",0.990869,1.0
7,4649,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера А",0.990053,1.0
8,124415,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2",1.000000,0.0
9,100211,"посёлок Песочный, Речная улица, дом 28, литера Б",1.000000,1.0


In [216]:
file_uploaded = 'test.csv'
pd.read_csv(file_uploaded, index_col=0).values

array([['город Пушкин, Кедринская улица, дом 12'],
       ['поселок Ушково, Пляжевая улица, дом 4Б'],
       ['г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е'],
       ['поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2'],
       ['поселок Песочный, Речная улица, дом 28, литера Б'],
       ['г.Санкт-Петербург, Большой проспект П.С., дом 49/18, литера Д'],
       ['поселок Солнечное, Комсомольская улица, дом 15, литера Б'],
       ['г.Санкт-Петербург, набережная Обводного канала, дом 223-225, литера П'],
       ['поселок Александровская, Беличья улица, дом 7'],
       ['поселок Парголово, Осиновая Роща, Вокзальное шоссе, дом 55, корпус 2'],
       ['г.Санкт-Петербург, Малоохтинский проспект, дом 64, литера В']],
      dtype=object)

In [211]:
train['full_address_building'].loc[:10].to_csv("test.csv")

In [119]:
def slow_get_best_matches(query, top_n, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = embeddings.dot(query_embedding.T).ravel()
    best = np.argpartition(scores, -top_n)[-top_n:]
    sentences['scores'] = scores
    sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'address'].apply(
        lambda x: lev_distance(query, x))
    best_matches = sentences[['id_building', 'address', 'scores', 'lev_distance']].loc[list(best)]
    add_data = pd.read_pickle("../data/processed/main_data.pkl")
    best_matches = best_matches.merge(add_data[['id_building', 'liter_building', 'name_district_full', 'name_town']])
    best_matches_within_id_indices = best_matches.groupby(['id_building']).scores.transform(max) == best_matches.scores
    best_matches = best_matches.loc[best_matches_within_id_indices].sort_values(by='scores',
                                                                                ascending=False).reset_index(drop=True)

    # add logic to topn selection
    # add catboost here to ass probabilities
    if len(best_matches[best_matches['scores'] >= 0.98]):
        return best_matches[best_matches['scores'] >= 0.98]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    else:
        return best_matches

In [120]:
slow_get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance,liter_building,name_district_full,name_town
0,84908,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",1.000000,65.0,М,Адмиралтейский район,г.Санкт-Петербург
1,31542,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера А",0.990537,65.0,А,Адмиралтейский район,г.Санкт-Петербург


# Step 4 NER + 1 one ML ALgo

In [5]:
from pullenti_wrapper.langs import (
    set_langs,
    RU
)
set_langs([RU])
from pullenti_wrapper.processor import (
    Processor,
    GEO,
    ADDRESS
)
from pullenti_wrapper.referent import Referent

In [6]:
processor = Processor([GEO, ADDRESS])

In [7]:
# recursive function
def get_ner_elements(referent, level=0):
    tmp = {}
    a = ""
    b = ""
    for key in referent.__shortcuts__:
        value = getattr(referent, key)
        if value in (None, 0, -1):
            continue
        if isinstance(value, Referent):
            get_ner_elements(value, level + 1)
        else:
            if key == 'type':
                a = value
            if key == 'name':
                b = value
                # print('ok', value)
            if key == 'house':
                a = "дом"
                b = value
                tmp[a] = b
            if key == 'flat':
                a = "квартира"
                b = value
                # print('ok', value)
                tmp[a] = b
            if key == 'building':
                a = "Литера"
                b = value
                tmp[a] = b
        if key == 'corpus':
            a = "корпус"
            b = value
            tmp[a] = b
    tmp[a] = b
    addr.append(tmp)
    return addr

In [8]:
addr = []
result = processor('город Пушкин, Екатерининский парк, дом б/н, литера Ц')
referent = result.matches[0].referent
x = get_ner_elements(referent)
print(addr)
print(x)

[{'город': 'ПУШКИН'}, {'парк': 'ЕКАТЕРИНИНСКИЙ'}, {'дом': '0', 'Литера': 'Ц'}]
[{'город': 'ПУШКИН'}, {'парк': 'ЕКАТЕРИНИНСКИЙ'}, {'дом': '0', 'Литера': 'Ц'}]


In [9]:
def get_address_ner_objects(address: str):
    global addr  # we need to declare global variable to change it
    processor = Processor([GEO, ADDRESS])
    addr = []
    try:
        result = processor(address)
        referent = result.matches[0].referent
        ner_model_res = get_ner_elements(referent)
        merged_data = {k: v for d in ner_model_res for k, v in d.items()}
        data = pd.DataFrame([merged_data])
        return merged_data
    except:
        return []

In [10]:
xx = get_address_ner_objects('посёлок Ушково, Пляжевая улица') #город Москва, Екатерининский парк, дом б/н, литера Ц
xx

{'поселок': 'УШКОВО', 'улица': 'ПЛЯЖЕВАЯ'}

In [14]:
# 
sandbox = train[:100]
# sandbox
sandbox['kek'] = sandbox.parallel_apply(lambda x: get_address_ner_objects(x['full_address_building']), axis=1) #get_address_ner_objects(

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field,miscellaneous_objects,kek
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN,35,Пушкинский,True,True,11132,28.0,4801.0,NaN,NaN,"город Пушкин, Кедринская улица","г.Пушкин, Кедринская ул.","'город':1,5 'кедринск':7 'кедринская':3 'пушкин':2,6 'улиц':8 'улица':4",True,True,15.0,True,28.0,город Пушкин,г.Пушкин,"'город':1,3 'пушкин':2,4",True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пушкинский район,"город Пушкин, Кедринская улица, дом 12, Пушкинский район","[{'город': 'ПУШКИН'}, {'улица': 'КЕДРИНСКАЯ'}, {'дом': '12'}]","{'город': 'ПУШКИН', 'улица': 'КЕДРИНСКАЯ', 'дом': '12'}"
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN,38,Курортный,True,True,6987,46.0,3.0,NaN,NaN,"посёлок Ушково, Пляжевая улица","пос. Ушково, Пляжевая ул.",'пляжев':7 'пляжевая':3 'поселок':5 'посёлок':1 'улиц':8 'улица':4 'ушков':6 'ушково':2,True,True,15.0,True,46.0,посёлок Ушково,пос. Ушково,'поселок':3 'посёлок':1 'ушков':4 'ушково':2,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Ушково, Пляжевая улица, дом 4Б, Курортный район","[{'поселок': 'УШКОВО'}, {'улица': 'ПЛЯЖЕВАЯ'}, {'дом': '4Б'}]","{'поселок': 'УШКОВО', 'улица': 'ПЛЯЖЕВАЯ', 'дом': '4Б'}"
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, к. 2, л. Е",198303,NaN,15,Кировский,True,True,6469,36.0,2194.0,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова","г.Санкт-Петербург, пр. Маршала Жукова","'г':1,8 'жуков':14 'жукова':7 'марша':13 'маршала':6 'петербург':4,11 'проспект':5,12 'санкт':3,10 'санкт-петербург':2,9",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кировский район,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е, Нежилое, Кировский район","[{'город': 'САНКТ-ПЕТЕРБУРГ'}, {'проспект': 'МАРШАЛА ЖУКОВА'}, {'дом': '30', 'корпус': '2', 'Литера': 'Е'}]","{'город': 'САНКТ-ПЕТЕРБУРГ', 'проспект': 'МАРШАЛА ЖУКОВА', 'дом': '30', 'корпус': '2', 'Литера': 'Е'}"
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, к. 2",NaN,NaN,38,Курортный,True,True,7838,37.0,319.0,NaN,161.0,"посёлок Белоостров, Дюны, Центральная улица","пос. Белоостров, Дюны, Центральная 

In [15]:
sandbox

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field,miscellaneous_objects,kek
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN,35,Пушкинский,True,True,11132,28.0,4801.0,NaN,NaN,"город Пушкин, Кедринская улица","г.Пушкин, Кедринская ул.","'город':1,5 'кедринск':7 'кедринская':3 'пушкин':2,6 'улиц':8 'улица':4",True,True,15.0,True,28.0,город Пушкин,г.Пушкин,"'город':1,3 'пушкин':2,4",True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пушкинский район,"город Пушкин, Кедринская улица, дом 12, Пушкинский район","[{'город': 'ПУШКИН'}, {'улица': 'КЕДРИНСКАЯ'}, {'дом': '12'}]","{'город': 'ПУШКИН', 'улица': 'КЕДРИНСКАЯ', 'дом': '12'}"
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN,38,Курортный,True,True,6987,46.0,3.0,NaN,NaN,"посёлок Ушково, Пляжевая улица","пос. Ушково, Пляжевая ул.",'пляжев':7 'пляжевая':3 'поселок':5 'посёлок':1 'улиц':8 'улица':4 'ушков':6 'ушково':2,True,True,15.0,True,46.0,посёлок Ушково,пос. Ушково,'поселок':3 'посёлок':1 'ушков':4 'ушково':2,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Ушково, Пляжевая улица, дом 4Б, Курортный район","[{'поселок': 'УШКОВО'}, {'улица': 'ПЛЯЖЕВАЯ'}, {'дом': '4Б'}]","{'поселок': 'УШКОВО', 'улица': 'ПЛЯЖЕВАЯ', 'дом': '4Б'}"
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, к. 2, л. Е",198303,NaN,15,Кировский,True,True,6469,36.0,2194.0,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова","г.Санкт-Петербург, пр. Маршала Жукова","'г':1,8 'жуков':14 'жукова':7 'марша':13 'маршала':6 'петербург':4,11 'проспект':5,12 'санкт':3,10 'санкт-петербург':2,9",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кировский район,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е, Нежилое, Кировский район","[{'город': 'САНКТ-ПЕТЕРБУРГ'}, {'проспект': 'МАРШАЛА ЖУКОВА'}, {'дом': '30', 'корпус': '2', 'Литера': 'Е'}]","{'город': 'САНКТ-ПЕТЕРБУРГ', 'проспект': 'МАРШАЛА ЖУКОВА', 'дом': '30', 'корпус': '2', 'Литера': 'Е'}"
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, к. 2",NaN,NaN,38,Курортный,True,True,7838,37.0,319.0,NaN,161.0,"посёлок Белоостров, Дюны, Центральная улица","пос. Белоостров, Дюны, Центральная 

In [16]:
sandbox['kek'].apply(pd.Series)
pd.concat([sandbox.drop(['kek'], axis=1), sandbox['kek'].apply(pd.Series)], axis=1)

,город,улица,дом,поселок,проспект,корпус,Литера,муниципальный район,набережная,шоссе,парк,переулок,площадь,аллея,,линия,автодорога
0,ПУШКИН,КЕДРИНСКАЯ,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ПЛЯЖЕВАЯ,4Б,УШКОВО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,САНКТ-ПЕТЕРБУРГ,NaN,30,NaN,МАРШАЛА ЖУКОВА,2,Е,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ЦЕНТРАЛЬНАЯ,5,БЕЛООСТРОВ,NaN,2,NaN,ДЮНЫ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,РЕЧНАЯ,28,ПЕСОЧНЫЙ,NaN,NaN,Б,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,САНКТ-ПЕТЕРБУРГ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,КОМСОМОЛЬСКАЯ,15,СОЛНЕЧНОЕ,NaN,NaN,Б,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,САНКТ-ПЕТЕРБУРГ,NaN,223/225,NaN,NaN,NaN,П,NaN,ОБВОДНОГО КАНАЛА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,БЕЛИЧЬЯ,7,АЛЕКСАНДРОВСКАЯ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,55,ПАРГОЛОВО,NaN,2,NaN,ОСИНОВАЯ РОЩА,NaN,ВОКЗАЛЬНОЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pd.concat([sandbox.drop(['kek'], axis=1), sandbox['kek'].apply(pd.Series)], axis=1)

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field,miscellaneous_objects,город,улица,дом,поселок,проспект,корпус,Литера,муниципальный район,набережная,шоссе,парк,переулок,площадь,аллея,,линия,автодорога
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN,35,Пушкинский,True,True,11132,28.0,4801.0,NaN,NaN,"город Пушкин, Кедринская улица","г.Пушкин, Кедринская ул.","'город':1,5 'кедринск':7 'кедринская':3 'пушкин':2,6 'улиц':8 'улица':4",True,True,15.0,True,28.0,город Пушкин,г.Пушкин,"'город':1,3 'пушкин':2,4",True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пушкинский район,"город Пушкин, Кедринская улица, дом 12, Пушкинский район","[{'город': 'ПУШКИН'}, {'улица': 'КЕДРИНСКАЯ'}, {'дом': '12'}]",ПУШКИН,КЕДРИНСКАЯ,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN,38,Курортный,True,True,6987,46.0,3.0,NaN,NaN,"посёлок Ушково, Пляжевая улица","пос. Ушково, Пляжевая ул.",'пляжев':7 'пляжевая':3 'поселок':5 'посёлок':1 'улиц':8 'улица':4 'ушков':6 'ушково':2,True,True,15.0,True,46.0,посёлок Ушково,пос. Ушково,'поселок':3 'посёлок':1 'ушков':4 'ушково':2,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Ушково, Пляжевая улица, дом 4Б, Курортный район","[{'поселок': 'УШКОВО'}, {'улица': 'ПЛЯЖЕВАЯ'}, {'дом': '4Б'}]",NaN,ПЛЯЖЕВАЯ,4Б,УШКОВО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, к. 2, л. Е",198303,NaN,15,Кировский,True,True,6469,36.0,2194.0,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова","г.Санкт-Петербург, пр. Маршала Жукова","'г':1,8 'жуков':14 'жукова':7 'марша':13 'маршала':6 'петербург':4,11 'проспект':5,12 'санкт':3,10 'санкт-петербург':2,9",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кировский район,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е, Нежилое, Кировский район","[{'город': 'САНКТ-ПЕТЕРБУРГ'}, {'проспект': 'МАРШАЛА ЖУКОВА'}, {'дом': '30', 'корпус': '2', 'Литера': 'Е'}]",САНКТ-ПЕТЕРБУРГ,NaN,30,NaN,МАРШАЛА ЖУКОВА,2,Е,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, к

In [321]:
# list = []
# for el in xx:
#     if el not in list:
#         list.append(el)

In [322]:
# list

['дом', 'Литера']

In [353]:
pd.DataFrame(xx)

,город,парк,дом,Литера
0,МОСКВА,NaN,NaN,NaN
1,NaN,ЕКАТЕРИНИНСКИЙ,NaN,NaN
2,NaN,NaN,0,Ц


test:
г.Санкт-Петербург, дом 223-225, наб. Обводного канала
г.Санкт-Петербург, дом 223-225
посёлок Солнечное, Комсомольская улица
г.Санкт-Петербург, проспект Динамо, дом 12

In [ ]:
# create test dataframe